# Introduction

This report is based on a TMDB Movies dataset as the selected choice to analyse about 10,000 movies collected from The Movie Database (TMDb), including user ratings and revenue. With the aim to answer questions such as:

- Which genres are most popular from year to year? 
- What kinds of properties are associated with movies that have high revenues?

Try to build your report around the analysis of at least one dependent variable and 
three independent variables.
make sure you familiarize yourself with the variables and the dataset context for ideas 
of what to explore.

In [2]:
# Imports statements for all of the packages to be use. Including magic word for plot:

import pandas as pd
import numpy as np
%matplotlib


Using matplotlib backend: MacOSX


In [3]:
# Loads CSV file:

filename: str = 'tmdb-movies.csv'
df = pd.read_csv(filename)

df.info()  # Allows to see which columns have missing values()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8042 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

(10866, 21)

In [4]:
# Trims and cleans data set for analysis:

to_drop = ['imdb_id','cast','homepage', 'director', 'tagline', 'keywords','overview','production_companies']
df.drop(to_drop, inplace= True, axis=1) # Drops columns that are not useful for analysis and updates dataframe
df.reset_index(level=0, drop=True)  # Resets dataframe index
df.shape  #shows dataframe dimension after previous changes

(10866, 13)

In [5]:
# Changes data types:

df['popularity'] = df['popularity'].astype(np.int64)
df[df['genres'].apply(lambda x: type(x)==str)]
df['release_date'] = pd.to_datetime(df['release_date'])
df['budget_adj'] = df['budget'].astype(np.int64)
df['revenue_adj'] = df['revenue'].astype(np.int64)

df.head()

,id,popularity,budget,revenue,original_title,runtime,genres,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,32,150000000,1513528810,Jurassic World,124,Action|Adventure|Science Fiction|Thriller,2015-06-09,5562,6.5,2015,150000000,1513528810
1,76341,28,150000000,378436354,Mad Max: Fury Road,120,Action|Adventure|Science Fiction|Thriller,2015-05-13,6185,7.1,2015,150000000,378436354
2,262500,13,110000000,295238201,Insurgent,119,Adventure|Science Fiction|Thriller,2015-03-18,2480,6.3,2015,110000000,295238201
3,140607,11,200000000,2068178225,Star Wars: The Force Awakens,136,Action|Adventure|Science Fiction|Fantasy,2015-12-15,5292,7.5,2015,200000000,2068178225
4,168259,9,190000000,1506249360,Furious 7,137,Action|Crime|Thriller,2015-04-01,2947,7.3,2015,190000000,1506249360


In [6]:
# Data Wrangling: Checking for cleanliness.

# Checks columns with NaN cells:
null_columns= df.columns[df.isnull().any()]

print('Columns with NaN cells', df[null_columns].isnull().sum())
df.shape  #shows dataframe dimension

Columns with NaN cells genres    23
dtype: int64


(10866, 13)

In [7]:
# Drops rows with NaN values:

NaN_row = df.isnull().sum().sum()
df= df.dropna()
NaN_row2 = df.isnull().sum().sum()

print('{} NaNs before cleanup and {} after.'.format(NaN_row,NaN_row2))
df.shape


23 NaNs before cleanup and 0 after.


(10843, 13)

In [8]:
# Checks and Drops duplicates:

dup_bef = sum(df.duplicated())

df.drop_duplicates(keep=False, inplace=True) # Drops duplicates and update dataframe
dup_aft = sum(df.duplicated())

print('{} Duplicate(s) before cleaning  and {} after.'.format(dup_bef,dup_aft))


1 Duplicate(s) before cleaning  and 0 after.


In [9]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10841 entries, 0 to 10865
Data columns (total 13 columns):
id                10841 non-null int64
popularity        10841 non-null int64
budget            10841 non-null int64
revenue           10841 non-null int64
original_title    10841 non-null object
runtime           10841 non-null int64
genres            10841 non-null object
release_date      10841 non-null datetime64[ns]
vote_count        10841 non-null int64
vote_average      10841 non-null float64
release_year      10841 non-null int64
budget_adj        10841 non-null int64
revenue_adj       10841 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(9), object(2)
memory usage: 1.2+ MB


(10841, 13)

In [10]:
# Index dataframe by popularity, genres and tittle to create a pivot view
df = df.sort_values(by=['popularity','genres','original_title'])

idx= ['popularity','original_title','genres']
multi_indexed_df = df.set_index(idx)
multi_indexed_df.head()


id    budget  \
popularity original_title                            genres                     
0          44 Minutes: The North Hollywood Shoot-Out Action   14842  10000000   
           A Good Man                                Action  286668   7000000   
           Absolute Deception                        Action  199556   5000000   
           Agent Cody Banks                          Action   10923  28000000   
           Alatriste                                 Action   13495  24000000   

                                                              revenue  \
popularity original_title                            genres             
0          44 Minutes: The North Hollywood Shoot-Out Action         0   
           A Good Man                                Action         0   
           Absolute Deception                        Action         0   
           Agent Cody Banks                          Action  58795814   
           Alatriste                                 Action  23321954   

                                                             runtime  \
popularity original_title                            genres            
0          44 Minutes: The North Hollywood Shoot-Out Action      103   
           A Good Man                                Action      100   
           Absolute Deception                        Action       92   
           Agent Cody Banks                          Action      102   
           Alatriste                                 Action      145   

                                                            release_date  \
popularity original_title                            genres                
0          44 Minutes: The North Hollywood Shoot-Out Action   2003-01-01   
           A Good Man                                Action   2014-08-19   
           Absolute Deception                        Action   2013-06-10   
           Agent Cody Banks                          Action   2003-03-12   
           Alatriste                                 Action   2006-09-01   

                                                             vote_count  \
popularity original_title                            genres               
0          44 Minutes: The North Hollywood Shoot-Out Action          13   
           A Good Man                                Action          26   
           Absolute Deception                        Action          14   
           Agent Cody Banks                          Action         161   
           Alatriste                                 Action          31   

                                                             vote_average  \
popularity original_title                            genres                 
0          44 Minutes: The North Hollywood Shoot-Out Action           6.2   
           A Good Man                                Action           4.9   
           Absolute Deception                        Action           4.4   
           Agent Cody Banks                          Action           4.9   
           Alatriste                                 Action           5.7   

                                                             release_year  \
popularity original_title                            genres                 
0          44 Minutes: The North Hollywood Shoot-Out Action          2003   
           A Good Man                                Action          2014   
           Absolute Deception                        Action          2013   
           Agent Cody Banks                          Action          2003   
           Alatriste                                 Action          2006   

                                                             budget_adj  \
popularity original_title                            genres               
0          44 Minutes: The North Hollywood Shoot-Out Action    10000000   
           A Good Man                                Action     7000000   
           Absolute Deception            

In [11]:
multi_indexed_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10841 entries, (0, 44 Minutes: The North Hollywood Shoot-Out, Action) to (32, Jurassic World, Action|Adventure|Science Fiction|Thriller)
Data columns (total 10 columns):
id              10841 non-null int64
budget          10841 non-null int64
revenue         10841 non-null int64
runtime         10841 non-null int64
release_date    10841 non-null datetime64[ns]
vote_count      10841 non-null int64
vote_average    10841 non-null float64
release_year    10841 non-null int64
budget_adj      10841 non-null int64
revenue_adj     10841 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 998.5+ KB


In [12]:
movie_list = [
    {'name': 'Movie 1',
    'genre' :'Action, Fantasy, Horror'},
    {'name': 'Movie 2',
    'genre' :'Action, Comedy, Family'},
    {'name': 'Movie 3',
    'genre' :'Biography, Drama'},
    {'name': 'Movie 4',
    'genre' :'Biography, Drama, Romance'},
    {'name': 'Movie 5',
    'genre' :'Drama'},
    {'name': 'Movie 6',
    'genre' :'Documentary'},
]

def reformat_movie_list(movies):
    name = []
    genre = []
    result = pd.DataFrame()
    for movie in movies:
        movie_name = movie["name"]
        movie_genres = movie["genre"].split(",")
        for movie_genre in movie_genres:
            name.append(movie_name.strip())
            genre.append(movie_genre.strip())
    result["name"] = name
    result["genre"] = genre
    return result

reformat_movie_list(movie_list)

,name,genre
0,Movie 1,Action
1,Movie 1,Fantasy
2,Movie 1,Horror
3,Movie 2,Action
4,Movie 2,Comedy
5,Movie 2,Family
6,Movie 3,Biography
7,Movie 3,Drama
8,Movie 4,Biography
9,Movie 4,Drama


In [13]:

def reformat_movie_list(movies):
    name_list = []
    genre_list = []
    result = pd.DataFrame()
    
    for i, movie in movies:
        movie_name = movie["original_title"]
        for agenre in movie["genres"].split("|"):
            name_list.append(movie_name)
            genre_list.append(agenre)
           
    result["name"] = name_list
    result["genre"] = genre_list
    
    return(result)


var = reformat_movie_list(df.iterrows())
print(var.head())

        

                                        name   genre
0  44 Minutes: The North Hollywood Shoot-Out  Action
1                                 A Good Man  Action
2                         Absolute Deception  Action
3                           Agent Cody Banks  Action
4                                  Alatriste  Action


In [14]:

var.(reformat_movie_list(df.iterrows())
            

SyntaxError: invalid syntax (<ipython-input-14-dcdb0daeb121>, line 2)

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.


> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!